## Морфологический анализ

Задачи морфологического анализа:

* Разбор слова — определение нормальной формы (леммы), основы (стема) и грамматических характеристик слова
* Синтез словоформы — генерация словоформы по заданным грамматическим характеристикам из леммы

Морфологический анализ — не самая сильная сторона NLTK.  Для этих задач лучше использовать `pymorphy2` и `pymystem3` для русского языка и, например, `Spacy` для европейских.

## Лемматизация

**Лемматизация** — процесс приведения словоформы к лемме, т.е. нормальной (словарной) форме. Это более сложная задача, чем стемминг, но и результаты дает гораздо более осмысленные, особенно для языков с богатой морфологией.

* кошке, кошку, кошкам, кошкой $\implies$ кошка
* бежал, бежит, бегу $\implies$  бежать
* белому, белым, белыми $\implies$ белый

## POS-tagging

**Частеречная разметка**, или **POS-tagging** _(part of speech tagging)_ —  определение части речи и грамматических характеристик слов в тексте (корпусе) с приписыванием им соответствующих тегов.

Для большинства слов возможно несколько разборов (т.е. несколько разных лемм, несколько разных частей речи и т.п.). Теггер генерирует  все варианты, ранжирует их по вероятности и по умолчанию выдает наиболее вероятный. Выбор одного разбора из нескольких называется **снятием омонимии**, или **дизамбигуацией**.

### Наборы тегов

Существует множество наборов грамматических тегов, или тегсетов:
* НКРЯ
* Mystem
* UPenn
* OpenCorpora (его использует pymorphy2)
* Universal Dependencies
* ...

Есть даже [библиотека](https://github.com/kmike/russian-tagsets) для преобразования тегов из одной системы в другую для русского языка, `russian-tagsets`. Но важно помнить, что любое такое преобразование будет с потерями!

На данный момент стандартом является **Universal Dependencies**. Подробнее про проект можно почитать [вот тут](http://universaldependencies.org/), а про теги — [вот тут](http://universaldependencies.org/u/pos/). Вот список основных (частереных) тегов UD:

* ADJ: adjective
* ADP: adposition
* ADV: adverb
* AUX: auxiliary
* CCONJ: coordinating conjunction
* DET: determiner
* INTJ: interjection
* NOUN: noun
* NUM: numeral
* PART: particle
* PRON: pronoun
* PROPN: proper noun
* PUNCT: punctuation
* SCONJ: subordinating conjunction
* SYM: symbol
* VERB: verb
* X: other

### pymystem3

**pymystem3** — это питоновская обертка для морфологичекого анализатора Mystem, сделанного в Яндексе. Его можно скачать отдельно и использовать из консоли. Отдельный плюс Mystem - он умеет разрешать омонимию (выбирает более релевантный вариант разбора слова для данного контекста).

* [Документация Mystem](https://tech.yandex.ru/mystem/doc/index-docpage/)
* [Документация pymystem3](http://pythonhosted.org/pymystem3/)

Инициализируем Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin - путь к `mystem`, если их несколько
* grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
* disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
* entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

In [28]:
!pip install --user pymystem3

In [29]:
text ='''
Театр "Компас".
Безымянная звезда.

Была на спектакле "Безымянная звезда" 31.01.2020 - смотрела впервые этот спектакль.
Понравилась сама постановка, единственное показалось в первом действии слегка затянутый сюжет,
а со второго действия было очень интересно.
Спасибо актёрам за игру. Есть интерес и дальше знакомиться с остальными спектаклями театра.

Ника написал(а)  07 февраля
'''

In [30]:
from pymystem3 import Mystem



In [31]:
tokens=['театр', 'компас', 'безымянная', 'звезда', 'спектакле', 'безымянная', 'звезда', 'смотрела', 'впервые', 'понравилась', 'сама', 'постановка', 'единственное', 'показалось', 'первом', 'действии', 'слегка', 'затянутый', 'сюжет', 'второго', 'действия', 'очень', 'спасибо', 'актёрам', 'интерес', 'дальше', 'знакомиться', 'остальными', 'спектаклями', 'ника', 'написал', 'февраля']

m = Mystem()
lemmas = m.lemmatize(' '.join(tokens))
print(lemmas)

['театр', ' ', 'компас', ' ', 'безымянный', ' ', 'звезда', ' ', 'спектакль', ' ', 'безымянный', ' ', 'звезда', ' ', 'смотреть', ' ', 'впервые', ' ', 'понравиться', ' ', 'сам', ' ', 'постановка', ' ', 'единственный', ' ', 'показываться', ' ', 'первый', ' ', 'действие', ' ', 'слегка', ' ', 'затягивать', ' ', 'сюжет', ' ', 'второй', ' ', 'действие', ' ', 'очень', ' ', 'спасибо', ' ', 'актер', ' ', 'интерес', ' ', 'далеко', ' ', 'знакомиться', ' ', 'остальной', ' ', 'спектакль', ' ', 'ника', ' ', 'написать', ' ', 'февраль', '\n']


In [32]:
parsed = m.analyze(text)
parsed[:10]

[{'text': '\n'},
 {'analysis': [{'lex': 'театр', 'wt': 1, 'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
  'text': 'Театр'},
 {'text': ' "'},
 {'analysis': [{'lex': 'компас', 'wt': 1, 'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
  'text': 'Компас'},
 {'text': '"'},
 {'text': '.'},
 {'text': '\n'},
 {'analysis': [{'lex': 'безымянный', 'wt': 1, 'gr': 'A=им,ед,полн,жен'}],
  'text': 'Безымянная'},
 {'text': ' '},
 {'analysis': [{'lex': 'звезда', 'wt': 0.690962732, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'звезда'}]

In [33]:
#части речи

for word in parsed[:20]:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        print(word['text'], pos)

Театр S
Компас S
Безымянная A
звезда S
Была V
на PR
спектакле S
Безымянная A


###  pymorphy

**pymorphy** — это полноценный морфологический анализатор, целиком написанный на Python.
В отличие от Mystem, он не учитывает контекст, а значит, вопрос разрешения омонимии надо будет решать самостоятельно.
Он также умеет ставить слова в нужную форму (спрягать и склонять).

[Документация pymorphy2](https://pymorphy2.readthedocs.io/en/latest/)

In [34]:
!pip install pymorphy3

In [35]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer(lang='ru')
p = morph.parse('запрограммировали')
print(p)

[Parse(word='запрограммировали', tag=OpencorporaTag('VERB,perf,tran plur,past,indc'), normal_form='запрограммировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'запрограммировали', 748, 4),))]


In [36]:
first = p[0]  # первый разбор
print('Слово:', first.word)
print('Тэг:', first.tag)
print('Лемма:', first.normal_form)
print('Вероятность:', first.score)

Слово: запрограммировали
Тэг: VERB,perf,tran plur,past,indc
Лемма: запрограммировать
Вероятность: 1.0


Из каждого тега можно достать более дробную информацию. Если граммема есть в разборе, то вернется ее значение, если ее нет, то вернется None. [Список граммем](https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html)

In [37]:
print(first.normalized)        # лемма
print(first.tag.POS)           # Part of Speech, часть речи
print(first.tag.animacy)       # одушевленность
print(first.tag.aspect)        # вид: совершенный или несовершенный
print(first.tag.case)          # падеж
print(first.tag.gender)        # род (мужской, женский, средний)
print(first.tag.involvement)   # включенность говорящего в действие
print(first.tag.mood)          # наклонение (повелительное, изъявительное)
print(first.tag.number)        # число (единственное, множественное)
print(first.tag.person)        # лицо (1, 2, 3)
print(first.tag.tense)         # время (настоящее, прошедшее, будущее)
print(first.tag.transitivity)  # переходность (переходный, непереходный)
print(first.tag.voice)         # залог (действительный, страдательный)

Parse(word='запрограммировать', tag=OpencorporaTag('INFN,perf,tran'), normal_form='запрограммировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'запрограммировать', 748, 0),))
VERB
None
perf
None
None
None
indc
plur
None
past
tran
None


In [38]:
print(first.normalized)
print(first.tag.POS)
print(first.tag.aspect)
print(first.tag.case)

Parse(word='запрограммировать', tag=OpencorporaTag('INFN,perf,tran'), normal_form='запрограммировать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'запрограммировать', 748, 0),))
VERB
perf
None


### mystem vs. pymorphy

1. Оба они могут работать с незнакомыми словами (out-of-vocabulary words, OOV).
2. *Скорость*. Mystem работает невероятно медленно под windows на больших текстах, но очень быстро, елси запускать из консоли в linux / mac os.
2. *Снятие омонимии*. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно не учитывает контекст

# Стандартная предобработка текста

In [39]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

# Набор данных с сайта lenta.ru
Корпус новостей с Lenta.Ru
Размер: 2 Гб

Количество новостей: 800K+

Период: Сентябрь 1999 - декабрь 2019

In [40]:
data = pd.read_csv('https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/lenta_small.csv', usecols=['text'])
data.tail(5)

,text
95,"Фото: Алексей Куденко / РИА Новости Название «Голландия» официально перестало существовать в качестве обозначения Нидерландов. Об этом сообщает местная радиостанция NOS. С 1 января 2020 года страна решила отказаться от использования названия «Голландия». Частные компании, посольства, министерства, школы и университеты будут использовать только официальное название для изменения имиджа страны. Реформа обошлась государству в 200 тысяч евро. О том, что Нидерланды отказались называть себя Голландией, сообщалось в октябре прошлого года. Власти посчитали, что Голландия ассоциируется у иностранных туристов с наркотиками и Кварталом красных фонарей (городской район, в котором процветает секс-индустрия — прим. «Ленты.ру»). Было предложено активно использовать в 2020 году название «Нидерланды» в..."
96,"Фото: Александр Вильф / РИА Новости Спикер парламента Эстонии Хенн Пыллуаас в своем новогоднем обращении к согражданам напомнил о территориальных претензиях к России. Текст обращения опубликован на сайте парламента. Пыллуаас отметил, что 2 февраля исполняется 100 лет со дня подписания Тартуского мирного договора, после чего Россия признала Эстонию, война за независимость закончилась и была установлена эстонско-российская граница. «После восстановления независимости все государства признали нас правопреемником Эстонской Республики во всех ее проявлениях, включая государственную границу. Тартуский мирный договор вступил в силу и все еще находится в реестре действующих международных договоров ООН», — добавил спикер парламента. Тартуский договор заключен между Советской Россией и Эстонией ..."
97,"Фото: Георгий Зимарев / РИА Новости Десять человек выжили после того, как военный вертолет Black Hawk с 13 высокопоставленными военными пропал в Тайване. Об этом сообщает местное издание Taiwan News. Среди выживших — начальник генерального штаба Шэнь Имин. Поиски остальных трех пассажиров продолжаются. На месте работают поисково-спасательные бригады. Отмечается, что около 9:00 вертолет совершил вынужденную посадку. Вертолет с военными Тайваня, в том числе начальником генерального штаба Шэнь Имином, пропал в четверг, 2 января, на севере острова. Связь с вертолетом пропала около 8:00 утра по местному времени. В декабре 2019 года, сообщалось, что вертолет Black Hawk Национальной гвардии американского штата Миннесота разбился недалеко от города Сент-Клауд. На борту воздушного судна находил..."
98,"Фото: Rahmat Gul / AP Photo По меньшей мере шесть человек погибли после того, как военный вертолет Black Hawk с военными совершил аварийную посадку на Тайване. Об этом сообщает Reuters со ссылкой на правительство страны. При этом в Минобороны Тайваня сообщают о восьми погибших, отмечает агентство. По данным South China Morning Post, среди погибших — начальник генерального штаба страны Шэнь Имин. Еще два человека, включая сотрудника Минобороны, числятся пропавшими без вести. Ранее сообщалось, что спасателям удалось обнаружить на месте аварии 10 выживших. Вертолет с 13 высокопоставленными военными Тайваня пропал в четверг, 2 января, на севере острова. Связь с вертолетом пропала около 8:00 утра по местному времени."
99,"Фото: Instagram / @irinashayk Российская супермодель Ирина Шейк поделилась с поклонниками пляжным снимком. Кадр опубликован в Instagram знаменитости. На фото 33-летняя Шейк позирует на камеру в бикини кислотно-розового цвета, расположившись на лежаке. Помимо этого, на ней надета кремовая панамка с нашивкой — популярный в 1990-х аксессуар — английского модного бренда Burberry. «День первый», — подписала снимок модель. Поклонники засыпали россиянку хвалебными комментариями. «Просто очень красива», «Мы уже ждем день второй!», «Ты мой идол в буквальном смысле слова! Я обожаю тебя», «Ты — самая красивая женщина в мире», — писали пользователи. В декабре 2019 года Ирина Шейк снялась для обложки модного журнала 7Hollywood в золотом металлическом корсете и чокере от французского модельера Тье..."


In [41]:
#Проверим на наличие пропусков
data["text"].isnull().value_counts()

,count
text,
False,99
True,1


In [42]:
#Удалим пропуски
data=data.dropna()
data["text"].isnull().value_counts()

,count
text,
False,99


In [43]:
# Приведение к нижнему регистру
data["text"]=data["text"].str.lower()
data.head()

<ipython-input-43-03f93db0a829>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"]=data["text"].str.lower()


,text
0,"фото: юрий мартьянов /« коммерсантъ» в россии работникам начнут выдавать электронные трудовые книжки вместо бумажных документов. соответствующий пакет законов вступил в силу в среду, 1 января, сообщается на официальном интернет-портале правовой информации. теперь работодатели смогут передавать в пенсионный фонд сведения о трудовой деятельности и стаже каждого работника в онлайн-режиме, а не по почте, как это было ранее. при этом работник сможет либо оставить старый документ, либо получить новую электронную трудовую книжку. для перехода на новую систему гражданин должен подать письменное заявление. те, кто не успеет сообщить о своем решении в 2020 году, смогут сделать это позже. однако те россияне, которые начнут официально работать после 1 января 2021 года, уже не будут иметь возможно..."
1,"фото: анатолий ломохов / globallookpress.com поклонников российской актрисы анастасии заворотнюк поздравили с новым годом и поблагодарили за поддержку. соответствующая запись появилась в instagram-аккаунте артистки. «всем вам настя передает огромный привет и благодарит за поддержку, за ваши молитвы и то душевное тепло, которое вы дарите», — говорится в публикации. также поклонникам актрисы пожелали здоровья и благополучия от заворотнюк и ее семьи. у 48-летней заворотнюк диагностировали рак мозга после того, как у нее начались сильные головные боли. эту информацию впервые подтвердили в октябре этого года. в настоящее время актриса проходит курс лечения. представители заворотнюк заявили, что не будут комментировать ее состояние."
2,"фото: rupert oberhäuser / globallookpress.com в россии вступил в силу закон об органической продукции питания, устанавливающий стандарты для производителей и продавцов. его авторы утвердили отличия органической еды от неорганической. документ был принят в августе 2018 года, однако участникам рынка был предоставлен переходный период до начала 2020 года. согласно ему, органической считается продукция, произведенная без применения пестицидов, удобрений, антибиотиков и стимуляторов роста для животных. также определяется, что органическая продукция не может производиться, храниться и перевозиться вместе с неорганической. производители могут добровольно сертифицировать свою продукцию для установления ее соответствия национальным и международным стандартам. созданный в рамках закона единый го..."
3,"фото: vladislav muslakov / unsplash спящие более 11 или менее четырех часов в сутки люди более подвержены фиброзу легких, чем те, кто спит примерно семь часов. к такому выводу пришла исследовательская группа с учеными из университетов манчестера, оксфорда, ньюкасла, университетских колледжей лондона и торонто, а также фонда nhs foundation. их результаты опубликовало издание medicalexpress. в исследовании использовались данные людей из британского биобанка. те, кто недосыпает и спит менее четырех часов, удваивают шансы на развитие фиброза легких. а спящие по 11 и более часов в сутки его утраивают. также повышенный риск на развитие болезни имеют те, кто ложится спать поздно ночью или работает посменно. отмечается, что из-за легочного фиброза в великобритании умирает около пяти тысяч чело..."
4,"фото: константин кокошкин / «коммерсантъ» российские театры и другие учреждения культуры освободили от уплаты налога на прибыль — 1 января вступили в силу соответствующие поправки в налоговый кодекс. поправки предусмотрены федеральным законом, принятым в июле 2019 года. в соответствии с ним, музеи, театры, библиотеки получили право применять нулевую ставку по налогу на прибыль при условии, что их учредителями выступают субъекты рф или муниципальные образования. льгота применяется в том случае, когда доходы от профильной деятельности составляют не менее 90 процентов от общей выручки театра, музея или библиотеки."


In [44]:
import re
# Функция для удаления пунктуации и цифр
def remove_punctuation_and_digits(text):
    # Удаляем всё, что не является буквой или пробелом
    return re.sub(r'[^а-яА-ЯёЁ\s]', '', text)

# Применение функции ко всем текстам
data['text'] = data['text'].apply(remove_punctuation_and_digits)
data.head()

<ipython-input-44-2ac51fc8aafd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(remove_punctuation_and_digits)


,text
0,фото юрий мартьянов коммерсантъ в россии работникам начнут выдавать электронные трудовые книжки вместо бумажных документов соответствующий пакет законов вступил в силу в среду января сообщается на официальном интернетпортале правовой информации теперь работодатели смогут передавать в пенсионный фонд сведения о трудовой деятельности и стаже каждого работника в онлайнрежиме а не по почте как это было ранее при этом работник сможет либо оставить старый документ либо получить новую электронную трудовую книжку для перехода на новую систему гражданин должен подать письменное заявление те кто не успеет сообщить о своем решении в году смогут сделать это позже однако те россияне которые начнут официально работать после января года уже не будут иметь возможности получить бумажную трудовую ...
1,фото анатолий ломохов поклонников российской актрисы анастасии заворотнюк поздравили с новым годом и поблагодарили за поддержку соответствующая запись появилась в аккаунте артистки всем вам настя передает огромный привет и благодарит за поддержку за ваши молитвы и то душевное тепло которое вы дарите говорится в публикации также поклонникам актрисы пожелали здоровья и благополучия от заворотнюк и ее семьи у летней заворотнюк диагностировали рак мозга после того как у нее начались сильные головные боли эту информацию впервые подтвердили в октябре этого года в настоящее время актриса проходит курс лечения представители заворотнюк заявили что не будут комментировать ее состояние
2,фото в россии вступил в силу закон об органической продукции питания устанавливающий стандарты для производителей и продавцов его авторы утвердили отличия органической еды от неорганической документ был принят в августе года однако участникам рынка был предоставлен переходный период до начала года согласно ему органической считается продукция произведенная без применения пестицидов удобрений антибиотиков и стимуляторов роста для животных также определяется что органическая продукция не может производиться храниться и перевозиться вместе с неорганической производители могут добровольно сертифицировать свою продукцию для установления ее соответствия национальным и международным стандартам созданный в рамках закона единый государственный реестр производителей органической продукции ...
3,фото спящие более или менее четырех часов в сутки люди более подвержены фиброзу легких чем те кто спит примерно семь часов к такому выводу пришла исследовательская группа с учеными из университетов манчестера оксфорда ньюкасла университетских колледжей лондона и торонто а также фонда их результаты опубликовало издание в исследовании использовались данные людей из британского биобанка те кто недосыпает и спит менее четырех часов удваивают шансы на развитие фиброза легких а спящие по и более часов в сутки его утраивают также повышенный риск на развитие болезни имеют те кто ложится спать поздно ночью или работает посменно отмечается что изза легочного фиброза в великобритании умирает около пяти тысяч человек в год примерно столько же становятся жертвами лейкемии фиброз легких р...
4,фото константин кокошкин коммерсантъ российские театры и другие учреждения культуры освободили от уплаты налога на прибыль января вступили в силу соответствующие поправки в налоговый кодекс поправки предусмотрены федеральным законом принятым в июле года в соответствии с ним музеи театры библиотеки получили право применять нулевую ставку по налогу на прибыль при условии что их учредителями выступают субъекты рф или муниципальные образования льгота применяется в том случае когда доходы от профильной деятельности составляют не менее процентов от общей выручки театра музея или библиотеки


In [45]:
#Лемматизация
from pymystem3 import Mystem

# Инициализация MyStem
mystem = Mystem()

# Функция для лемматизации текста
def lemmatize_text(text):
    # Лемматизация текста
    lemmas = mystem.lemmatize(text)
    # Объединение лемм в строку
    return ''.join(lemmas).strip()


In [46]:
# Применение лемматизации ко всем текстам
data['lemmatized_text'] = data["text"].apply(lemmatize_text)

In [47]:
data.head()

,text,lemmatized_text
0,фото юрий мартьянов коммерсантъ в россии работникам начнут выдавать электронные трудовые книжки вместо бумажных документов соответствующий пакет законов вступил в силу в среду января сообщается на официальном интернетпортале правовой информации теперь работодатели смогут передавать в пенсионный фонд сведения о трудовой деятельности и стаже каждого работника в онлайнрежиме а не по почте как это было ранее при этом работник сможет либо оставить старый документ либо получить новую электронную трудовую книжку для перехода на новую систему гражданин должен подать письменное заявление те кто не успеет сообщить о своем решении в году смогут сделать это позже однако те россияне которые начнут официально работать после января года уже не будут иметь возможности получить бумажную трудовую ...,фото юрий мартьянов коммерсант в россия работник начинать выдавать электронный трудовой книжка вместо бумажный документ соответствующий пакет закон вступать в сила в среда январь сообщаться на официальный интернетпортал правовой информация теперь работодатель смочь передавать в пенсионный фонд сведение о трудовой деятельность и стаж каждый работник в онлайнрежим а не по почта как это быть ранее при это работник смочь либо оставлять старый документ либо получать новый электронный трудовой книжка для переход на новый система гражданин должный подавать письменный заявление тот кто не успевать сообщать о свой решение в год смочь сделать это поздно однако тот россиянин который начинать официально работать после январь год уже не быть иметь возможность получать бумажный трудовой книжка...
1,фото анатолий ломохов поклонников российской актрисы анастасии заворотнюк поздравили с новым годом и поблагодарили за поддержку соответствующая запись появилась в аккаунте артистки всем вам настя передает огромный привет и благодарит за поддержку за ваши молитвы и то душевное тепло которое вы дарите говорится в публикации также поклонникам актрисы пожелали здоровья и благополучия от заворотнюк и ее семьи у летней заворотнюк диагностировали рак мозга после того как у нее начались сильные головные боли эту информацию впервые подтвердили в октябре этого года в настоящее время актриса проходит курс лечения представители заворотнюк заявили что не будут комментировать ее состояние,фото анатолий ломохов поклонник российский актриса анастасия заворотнюк поздравлять с новый год и поблагодарить за поддержка соответствующий запись появляться в аккаунт артистка весь вы настя передавать огромный привет и благодарить за поддержка за ваш молитва и то душевный тепло который вы дарить говориться в публикация также поклонник актриса пожелать здоровье и благополучие от заворотнюк и ее семья у летний заворотнюк диагностировать рак мозг после то как у она начинаться сильный головной боль этот информация впервые подтверждать в октябрь этот год в настоящий время актриса проходить курс лечение представитель заворотнюк заявлять что не быть комментировать ее состояние
2,фото в россии вступил в силу закон об органической продукции питания устанавливающий стандарты для производителей и продавцов его авторы утвердили отличия органической еды от неорганической документ был принят в августе года однако участникам рынка был предоставлен переходный период до начала года согласно ему органической считается продукция произведенная без применения пестицидов удобрений антибиотиков и стимуляторов роста для животных также определяется что органическая продукция не может производиться храниться и перевозиться вместе с неорганической производители могут добровольно сертифицировать свою продукцию для установления ее соответствия национальным и международным стандартам созданный в рамках закона единый государственный реестр производителей органической продукции ...,фото в россия вступать в сила закон об органический продукция питание устанавливать стандарт для производитель и продавец его автор утверждать отличие органический еда от неорганический документ быть принимать в август год од

In [48]:
import nltk
from nltk.corpus import stopwords

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Функция для удаления стоп-слов
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

# Удаление стоп-слов из лемматизированного текста
data['lemmatized_text'] = data['lemmatized_text'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
data.head()

,text,lemmatized_text
0,фото юрий мартьянов коммерсантъ в россии работникам начнут выдавать электронные трудовые книжки вместо бумажных документов соответствующий пакет законов вступил в силу в среду января сообщается на официальном интернетпортале правовой информации теперь работодатели смогут передавать в пенсионный фонд сведения о трудовой деятельности и стаже каждого работника в онлайнрежиме а не по почте как это было ранее при этом работник сможет либо оставить старый документ либо получить новую электронную трудовую книжку для перехода на новую систему гражданин должен подать письменное заявление те кто не успеет сообщить о своем решении в году смогут сделать это позже однако те россияне которые начнут официально работать после января года уже не будут иметь возможности получить бумажную трудовую ...,фото юрий мартьянов коммерсант россия работник начинать выдавать электронный трудовой книжка вместо бумажный документ соответствующий пакет закон вступать сила среда январь сообщаться официальный интернетпортал правовой информация работодатель смочь передавать пенсионный фонд сведение трудовой деятельность стаж каждый работник онлайнрежим почта это ранее это работник смочь либо оставлять старый документ либо получать новый электронный трудовой книжка переход новый система гражданин должный подавать письменный заявление успевать сообщать свой решение год смочь сделать это поздно однако россиянин который начинать официально работать январь год иметь возможность получать бумажный трудовой книжка декабрь сообщаться использование трудовой книжка электронный вид считать полезный процент росс...
1,фото анатолий ломохов поклонников российской актрисы анастасии заворотнюк поздравили с новым годом и поблагодарили за поддержку соответствующая запись появилась в аккаунте артистки всем вам настя передает огромный привет и благодарит за поддержку за ваши молитвы и то душевное тепло которое вы дарите говорится в публикации также поклонникам актрисы пожелали здоровья и благополучия от заворотнюк и ее семьи у летней заворотнюк диагностировали рак мозга после того как у нее начались сильные головные боли эту информацию впервые подтвердили в октябре этого года в настоящее время актриса проходит курс лечения представители заворотнюк заявили что не будут комментировать ее состояние,фото анатолий ломохов поклонник российский актриса анастасия заворотнюк поздравлять новый год поблагодарить поддержка соответствующий запись появляться аккаунт артистка весь настя передавать огромный привет благодарить поддержка ваш молитва душевный тепло который дарить говориться публикация также поклонник актриса пожелать здоровье благополучие заворотнюк семья летний заворотнюк диагностировать рак мозг начинаться сильный головной боль информация впервые подтверждать октябрь год настоящий время актриса проходить курс лечение представитель заворотнюк заявлять комментировать состояние
2,фото в россии вступил в силу закон об органической продукции питания устанавливающий стандарты для производителей и продавцов его авторы утвердили отличия органической еды от неорганической документ был принят в августе года однако участникам рынка был предоставлен переходный период до начала года согласно ему органической считается продукция произведенная без применения пестицидов удобрений антибиотиков и стимуляторов роста для животных также определяется что органическая продукция не может производиться храниться и перевозиться вместе с неорганической производители могут добровольно сертифицировать свою продукцию для установления ее соответствия национальным и международным стандартам созданный в рамках закона единый государственный реестр производителей органической продукции ...,фото россия вступать сила закон органический продукция питание устанавливать стандарт производитель продавец автор утверждать отличие органический еда неорганический документ принимать август год однако участник рынок предоставлять переходный период начало год согласно органический считаться продукция произв

# Задание

Проведлите предобработку для набора данных reviews_small.csv - отзывы на смартфоны.

1. Загрузите файл в свой репозиторий в папку data
2. Получите ссылку на файл для использования в Colab.
  - Нужно нажать на имя файла.
  - Откроется окно просмотра.
  - Нажимаем RAW
  - Откроется сам файл
  - Копируем ссылку из адресной строки браузера

In [50]:
df = pd.read_csv('https://raw.githubusercontent.com/setarets/mo2025/refs/heads/main/data/reviews_small.csv')
df.tail(5)

,review,rating
994,достоинства приятная внешность большой экран прочный нормально тянет игрули недостатки нетукомментарий берите,4
995,достоинства за такую сумму смартфон очень даже 2 гб оперативки аккумулятор 4000ма экран в 5 5 недостатки ips экран но это смотря кому каккомментарий за такую сумму и с такими параметрами смартфон не плохой советую,5
996,достоинства большой дисплей ёмкий аккумулятор довольно шустрый аппарат для своей цены недостатки отсутствие обновлений для android камера слабовата но что ждать от телефона за 7 тысяч рублей комментарий брал осознанно эту модель так как брат уже давно пользуется таким аппаратом и доволен соотношение цена качество идеальное основные задачи телефона выполняет хорошо интернет игры музыка видео всё без проблем ну и всё так брэнд по сравнению с мейзу хайскринами и прочей нечистью,4
997,достоинства неплохой ips экран hd разрешение отдельный слот для microsd карты наличие 4g относительно невысокая цена недостатки размер экрана 5 5 несколько великоват экран слепнет на солнце процессор не отличается высокой производительностью комментарий вполне неплохой аппарат если не планируется играть в тяжелые игрушки и не напрягает размер,4
998,достоинства порадовал аккумулятор на 4000 мач 4 х ядерный процессор 2 гб оперативки много встроенной памяти приятная цена камера недостатки старенький андроид на улице плохо видно что происходит на экране комментарий хороший аппарат за свои деньги цена полностью устраивает рекомендую к покупке,5


In [51]:
data["text"].isnull().value_counts()

,count
text,
False,99


In [52]:
df["review"]=df["review"].str.lower()
df.head()

,review,rating
0,достоинства удобно лежит в детской руке неплохой дизайн отличная задумка недостатки но реализация отвратительная глюк на глюке за два месяца использования столкнулись со следующими проблемами спервого дня использования 1 не видит симку приходится разбирать и колдовать поворачивая ее по полмиллиметра в разные стороны да слот под симку чуть больше симки 2 провод приходится перетыкать в гнездо по несколько раз чтобы пошла зарядка 3 родительский контроль работает через раз в частности контроль приложений чудачит как хочет то появляется значок блокировки приложения когда оно доступно то наоборот 4 настроить отслеживание местоположения так и не удалось несмотря на два дня танцев с бубнами ни мой телефон nexus 5x ни мужа iphone 4s так и не смогли подклю...,1
1,достоинства экран яркость недостатки всё остальное прошивка батарея не держала толком быстро вздулась пришлось править звук в инженерном меню оболочка для детей отвратительный нелогичный аляповато оформленный неудобный обрезок нормального андроида одна сим комментарий никогда не покупайте это убожество,1
2,достоинства для ребенка яркий телефон нормальный экран ребенку понравился недостатки не работает родительский контроль двое суток настраивали одни поработал и все эта функция не настраивается вообще в магазине не берут назад сказали сначала качество тел проведут а потом будут решать пользовались неделю,3
3,достоинства цена недостатки очень слабоживущий сенсор комментарий продержался 4 месяца умер сенсор при бережной эксплуатации за 45 дней по гарантии заменили сенсор якобы когда забирал проверил умер вечером в тот же день прямо он более китай чем сами китайцы для детей не берите лучше посмотрите дешевые модели известных брендов или фирменных китайцев,1
4,телефон как одноразовая бритва долго не работает корпус сделан из самого плохого и дешевого пластика пришло обновление программы и при обновлении телефон сразу глюканул так что сбросил все и больше не запускался после гарантийного ремонта через месяц при зарядке вздулась батарея и сломала крепление задней крышки а гарантия как раз закончилась вообщем разовый телефон не советую,1


In [53]:
import re
def remove_punctuation_and_digits(review):
    return re.sub(r'[^а-яА-ЯёЁ\s]', '', review)
df['review'] = df['review'].apply(remove_punctuation_and_digits)
df.head()

,review,rating
0,достоинства удобно лежит в детской руке неплохой дизайн отличная задумка недостатки но реализация отвратительная глюк на глюке за два месяца использования столкнулись со следующими проблемами спервого дня использования не видит симку приходится разбирать и колдовать поворачивая ее по полмиллиметра в разные стороны да слот под симку чуть больше симки провод приходится перетыкать в гнездо по несколько раз чтобы пошла зарядка родительский контроль работает через раз в частности контроль приложений чудачит как хочет то появляется значок блокировки приложения когда оно доступно то наоборот настроить отслеживание местоположения так и не удалось несмотря на два дня танцев с бубнами ни мой телефон ни мужа так и не смогли подключиться к устройству...,1
1,достоинства экран яркость недостатки всё остальное прошивка батарея не держала толком быстро вздулась пришлось править звук в инженерном меню оболочка для детей отвратительный нелогичный аляповато оформленный неудобный обрезок нормального андроида одна сим комментарий никогда не покупайте это убожество,1
2,достоинства для ребенка яркий телефон нормальный экран ребенку понравился недостатки не работает родительский контроль двое суток настраивали одни поработал и все эта функция не настраивается вообще в магазине не берут назад сказали сначала качество тел проведут а потом будут решать пользовались неделю,3
3,достоинства цена недостатки очень слабоживущий сенсор комментарий продержался месяца умер сенсор при бережной эксплуатации за дней по гарантии заменили сенсор якобы когда забирал проверил умер вечером в тот же день прямо он более китай чем сами китайцы для детей не берите лучше посмотрите дешевые модели известных брендов или фирменных китайцев,1
4,телефон как одноразовая бритва долго не работает корпус сделан из самого плохого и дешевого пластика пришло обновление программы и при обновлении телефон сразу глюканул так что сбросил все и больше не запускался после гарантийного ремонта через месяц при зарядке вздулась батарея и сломала крепление задней крышки а гарантия как раз закончилась вообщем разовый телефон не советую,1


In [54]:
from pymystem3 import Mystem
mystem = Mystem()
def lemmatize_text(text):
    lemmas = mystem.lemmatize(text)
    return ''.join(lemmas).strip()
    data['lemmatized_text'] = data["text"].apply(lemmatize_text)
data.head()

,text,lemmatized_text
0,фото юрий мартьянов коммерсантъ в россии работникам начнут выдавать электронные трудовые книжки вместо бумажных документов соответствующий пакет законов вступил в силу в среду января сообщается на официальном интернетпортале правовой информации теперь работодатели смогут передавать в пенсионный фонд сведения о трудовой деятельности и стаже каждого работника в онлайнрежиме а не по почте как это было ранее при этом работник сможет либо оставить старый документ либо получить новую электронную трудовую книжку для перехода на новую систему гражданин должен подать письменное заявление те кто не успеет сообщить о своем решении в году смогут сделать это позже однако те россияне которые начнут официально работать после января года уже не будут иметь возможности получить бумажную трудовую ...,фото юрий мартьянов коммерсант россия работник начинать выдавать электронный трудовой книжка вместо бумажный документ соответствующий пакет закон вступать сила среда январь сообщаться официальный интернетпортал правовой информация работодатель смочь передавать пенсионный фонд сведение трудовой деятельность стаж каждый работник онлайнрежим почта это ранее это работник смочь либо оставлять старый документ либо получать новый электронный трудовой книжка переход новый система гражданин должный подавать письменный заявление успевать сообщать свой решение год смочь сделать это поздно однако россиянин который начинать официально работать январь год иметь возможность получать бумажный трудовой книжка декабрь сообщаться использование трудовой книжка электронный вид считать полезный процент росс...
1,фото анатолий ломохов поклонников российской актрисы анастасии заворотнюк поздравили с новым годом и поблагодарили за поддержку соответствующая запись появилась в аккаунте артистки всем вам настя передает огромный привет и благодарит за поддержку за ваши молитвы и то душевное тепло которое вы дарите говорится в публикации также поклонникам актрисы пожелали здоровья и благополучия от заворотнюк и ее семьи у летней заворотнюк диагностировали рак мозга после того как у нее начались сильные головные боли эту информацию впервые подтвердили в октябре этого года в настоящее время актриса проходит курс лечения представители заворотнюк заявили что не будут комментировать ее состояние,фото анатолий ломохов поклонник российский актриса анастасия заворотнюк поздравлять новый год поблагодарить поддержка соответствующий запись появляться аккаунт артистка весь настя передавать огромный привет благодарить поддержка ваш молитва душевный тепло который дарить говориться публикация также поклонник актриса пожелать здоровье благополучие заворотнюк семья летний заворотнюк диагностировать рак мозг начинаться сильный головной боль информация впервые подтверждать октябрь год настоящий время актриса проходить курс лечение представитель заворотнюк заявлять комментировать состояние
2,фото в россии вступил в силу закон об органической продукции питания устанавливающий стандарты для производителей и продавцов его авторы утвердили отличия органической еды от неорганической документ был принят в августе года однако участникам рынка был предоставлен переходный период до начала года согласно ему органической считается продукция произведенная без применения пестицидов удобрений антибиотиков и стимуляторов роста для животных также определяется что органическая продукция не может производиться храниться и перевозиться вместе с неорганической производители могут добровольно сертифицировать свою продукцию для установления ее соответствия национальным и международным стандартам созданный в рамках закона единый государственный реестр производителей органической продукции ...,фото россия вступать сила закон органический продукция питание устанавливать стандарт производитель продавец автор утверждать отличие органический еда неорганический документ принимать август год однако участник рынок предоставлять переходный период начало год согласно органический считаться продукция произв

In [55]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])
data['lemmatized_text'] = data['lemmatized_text'].apply(remove_stopwords)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,lemmatized_text
0,фото юрий мартьянов коммерсантъ в россии работникам начнут выдавать электронные трудовые книжки вместо бумажных документов соответствующий пакет законов вступил в силу в среду января сообщается на официальном интернетпортале правовой информации теперь работодатели смогут передавать в пенсионный фонд сведения о трудовой деятельности и стаже каждого работника в онлайнрежиме а не по почте как это было ранее при этом работник сможет либо оставить старый документ либо получить новую электронную трудовую книжку для перехода на новую систему гражданин должен подать письменное заявление те кто не успеет сообщить о своем решении в году смогут сделать это позже однако те россияне которые начнут официально работать после января года уже не будут иметь возможности получить бумажную трудовую ...,фото юрий мартьянов коммерсант россия работник начинать выдавать электронный трудовой книжка вместо бумажный документ соответствующий пакет закон вступать сила среда январь сообщаться официальный интернетпортал правовой информация работодатель смочь передавать пенсионный фонд сведение трудовой деятельность стаж каждый работник онлайнрежим почта это ранее это работник смочь либо оставлять старый документ либо получать новый электронный трудовой книжка переход новый система гражданин должный подавать письменный заявление успевать сообщать свой решение год смочь сделать это поздно однако россиянин который начинать официально работать январь год иметь возможность получать бумажный трудовой книжка декабрь сообщаться использование трудовой книжка электронный вид считать полезный процент росс...
1,фото анатолий ломохов поклонников российской актрисы анастасии заворотнюк поздравили с новым годом и поблагодарили за поддержку соответствующая запись появилась в аккаунте артистки всем вам настя передает огромный привет и благодарит за поддержку за ваши молитвы и то душевное тепло которое вы дарите говорится в публикации также поклонникам актрисы пожелали здоровья и благополучия от заворотнюк и ее семьи у летней заворотнюк диагностировали рак мозга после того как у нее начались сильные головные боли эту информацию впервые подтвердили в октябре этого года в настоящее время актриса проходит курс лечения представители заворотнюк заявили что не будут комментировать ее состояние,фото анатолий ломохов поклонник российский актриса анастасия заворотнюк поздравлять новый год поблагодарить поддержка соответствующий запись появляться аккаунт артистка весь настя передавать огромный привет благодарить поддержка ваш молитва душевный тепло который дарить говориться публикация также поклонник актриса пожелать здоровье благополучие заворотнюк семья летний заворотнюк диагностировать рак мозг начинаться сильный головной боль информация впервые подтверждать октябрь год настоящий время актриса проходить курс лечение представитель заворотнюк заявлять комментировать состояние
2,фото в россии вступил в силу закон об органической продукции питания устанавливающий стандарты для производителей и продавцов его авторы утвердили отличия органической еды от неорганической документ был принят в августе года однако участникам рынка был предоставлен переходный период до начала года согласно ему органической считается продукция произведенная без применения пестицидов удобрений антибиотиков и стимуляторов роста для животных также определяется что органическая продукция не может производиться храниться и перевозиться вместе с неорганической производители могут добровольно сертифицировать свою продукцию для установления ее соответствия национальным и международным стандартам созданный в рамках закона единый государственный реестр производителей органической продукции ...,фото россия вступать сила закон органический продукция питание устанавливать стандарт производитель продавец автор утверждать отличие органический еда неорганический документ принимать август год однако участник рынок предоставлять переходный период начало год согласно органический считаться продукция произв